# Опорные векторы и коэффициент сходства

Для работы нужен файл clouds.data.h5. Если нет в папке - запустить 29_image_preprocessing.ipynb

1. Построить модель опорных векторов для типа облака Fish
2. Сделать оценку качества по F1 и коэфициенту сходства

## Подключение библиотек

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import f1_score

## Загрузка данных

In [3]:
clouds = pd.read_hdf("../data/out/clouds.data.h5")
print(clouds.head())

    0   1    2    3    4    5    6    7    8    9  ...  183745  183746  \
0   0   0    0    0    0    0    0    0    0    0  ...     102     113   
1  72  43   32   39   49   69   71   47   25   22  ...      38      58   
2  97  98  100  102  105  107  109  110  111  112  ...       0       0   
3  88  87   90   94   94   89   87   87   82   82  ...     166     164   
4  18  19   21   21   21   20   18   17   15   16  ...     125     137   

   183747  183748  183749        Image  \
0     102      96     117  0011165.jpg   
1      83     111     134  002be4f.jpg   
2       0       0       0  0031ae9.jpg   
3     109      70      72  0035239.jpg   
4      93      60      49  003994e.jpg   

                                                Fish  \
0  264918 937 266318 937 267718 937 269118 937 27...   
1  233813 878 235213 878 236613 878 238010 881 23...   
2  3510 690 4910 690 6310 690 7710 690 9110 690 1...   
3                                                NaN   
4  2367966 18 2367985 

## Оставить данные только по Fish

In [4]:
clouds.drop(
    labels=[
        "Image",
        "Flower",
        "Gravel",
        "Sugar",
    ],
    axis=1,
    inplace=True,
)

## Разделить данные

In [5]:
clouds_train, clouds_test = train_test_split(clouds, test_size=0.2)
del clouds
print(clouds_train.head())

        0    1    2    3    4    5    6    7    8    9  ...  183741  183742  \
2612  148  151  154  156  157  155  153  151  137  133  ...      77      78   
4836    0    0    1    1    1    1    1    1    7    0  ...      44      24   
4080  202  159  113   98  105   91  101  167  136  126  ...      73      61   
5541    9   35   51   47   50   61   55   37   24   63  ...     177     178   
2667  217  216  212  211  219  223  196  156  158  145  ...      59      80   

      183743  183744  183745  183746  183747  183748  183749  \
2612      78      78      76      77      78      79      79   
4836      27      23      25      21      20      21      20   
4080      75     126     145     143     149     131     125   
5541     177     174     177     171     159     157     166   
2667      82      91      88      78      55      40      45   

                                                   Fish  
2612  45095 367 46495 367 47895 367 49295 367 50695 ...  
4836  471292 508 472692 

## SVM

Считать последовательно коэффициенты для пакетов по 100 изображений

In [6]:
y = clouds_train["Fish"].notnull().astype("int8")
x = pd.DataFrame(clouds_train).drop(labels=["Fish"], axis=1)
model = SGDClassifier(loss="log_loss", warm_start=True)

In [7]:
for i in range(len(clouds_train) // 100):
    model.partial_fit(x[i*100:i*100+100], y[i*100:i*100+100], classes=[0,1])

In [8]:
del x
del y

## Средняя область облаков

В качестве локализации облаков на изображении возьмем среднюю область по обучающей выборке

In [9]:
image_x = 2100
image_x_4 = image_x // 4
image_y =1400
image_y_4 = image_y // 4

In [10]:
def locate_rectangle(a):
    vals = [int(i) for i in a[0].split(" ")]
    x = vals[0] // image_y
    y = vals[0] % image_y
    width = (image_x + (vals[-2] + vals[-1]) // image_y - x) % image_x
    height = (vals[-2] + vals[-1]) % image_y - y
    return [x, y, width, height]

In [11]:
areas = pd.DataFrame(clouds_train["Fish"].copy().dropna(axis=0))
areas = areas.apply(locate_rectangle, axis=1, result_type="expand")
coords = np.array(areas.mean() // 4)
print(coords)

[ 93. 103. 332. 110.]


In [12]:
sgd_mask = np.zeros(image_x_4 * image_y_4,
                    dtype="uint8").reshape(image_x_4, image_y_4)
for x in range(image_x_4):
    for y in range(image_y_4):
        if (x >= coords[0] and x <= (coords[0] + coords[3]) and
                y >= coords[1] and y <= (coords[1] + coords[3])):
            sgd_mask[x][y] = 1
sgd_mask = sgd_mask.reshape(image_x_4 * image_y_4)
print(sgd_mask.sum())

12321


## Предсказать значения

In [15]:
result = pd.DataFrame(
    {
        "EncodedPixels": clouds_test["Fish"],
        "Is_Fish": clouds_test["Fish"].notnull().astype("int8")
    }
)
result["target"] = model.predict(pd.DataFrame(clouds_test).drop(labels=["Fish"], axis=1))
print(result.head(10))

                                          EncodedPixels  Is_Fish  target
3052  1401 677 2801 677 4201 677 5601 677 7001 677 8...        1       1
1808  701772 505 703172 505 704572 505 705972 505 70...        1       1
3133  1563331 469 1564731 469 1566131 469 1567531 46...        1       1
112   40622 1369 42022 1369 43422 1369 44822 1369 46...        1       1
4856                                                NaN        0       1
2580  16239 489 17639 489 19039 489 20439 489 21839 ...        1       1
3377  16 1109 1416 1109 2816 1109 4216 1109 5616 110...        1       1
883                                                 NaN        0       1
4386  637620 773 639020 773 640420 773 641820 773 64...        1       1
538   604975 968 606375 968 607775 968 609175 968 61...        1       1


## Оценка предсказания F1

In [16]:
print(f'Опорные векторы: {round(f1_score(result["Is_Fish"], result["target"]), 3)}')
print(f'Все Fish: {round(f1_score(result["Is_Fish"], np.ones(len(result))), 3)}')

Опорные векторны: 0.647
Все Fish: 0.685


## Оценка по Дайсу

In [21]:
image_x = 525
image_y = 350


def mask_rate(a, x, y):
    b = a // 1400
    return np.round(x*(b*x // 2100) + y*(a%1400) // 1400).astype("uint32")

def calc_mask(px, x=525, y=350):
    p = np.array([int(n) for n in px.split(" ")]).reshape(-1, 2)
    mask = np.zeros(y*x, dtype="uint8")
    for i, m in p:
        mask[mask_rate(i, x, y)-1:mask_rate(m + i, x, y)] = 1
    return mask.reshape(y, x).transpose()

def calc_dice(x):
    dice = 0
    px = x["EncodedPixels"]
    if px != px and x["target"] == 0:
        dice = 1
    elif px == px and x["target"] == 1:
        mask = calc_mask(px).flatten()
        target = np.ones(image_x*image_y, dtype="uint8")
        dice = 2*np.sum(target[mask==1]) / (np.sum(target) + np.sum(mask))
    
    return dice

In [22]:
dice = result.apply(calc_dice, axis=1, result_type="expand")
print(f'Опорные векторы, Fish {round(dice.mean(), 3)}')
print(f'Нет облаков, Fish {round(len(result[result["Is_Fish"] == 0]) / len(result), 3)}')

Опорные векторы, Fish 0.241
Опорные векторы, Fish 0.479
